In [2]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [1]:
import torch
import random
from torchvision import transforms
import timm
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from PIL import Image
import os
import pandas as pd
import numpy as np
from albumentations.pytorch import ToTensorV2
import albumentations as A

import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report

In [3]:
import torch
import random
from torchvision import transforms
import timm
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from PIL import Image
import os
import pandas as pd
import numpy as np
from albumentations.pytorch import ToTensorV2
import albumentations as A

import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score, classification_report

# 랜덤시드 고정
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.cuda.empty_cache()

# 테스트 이미지 변환 설정
img_size = 380
# data_transform = A.Compose([
#     A.Resize(height=img_size, width=img_size), # 크기 조정
#     A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # 정규화
#     ToTensorV2() # 텐서로 변환
# ])

# TTA 변환 정의 - 1(kkh-7)
# tta_transforms = [
#     A.Compose([A.HorizontalFlip(p=1), A.GaussNoise(var_limit=(500, 500), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(45, 45), p=1), A.GaussNoise(var_limit=(500, 500), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(90, 90), p=1), A.GaussNoise(var_limit=(500, 500), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(135, 135), p=1), A.GaussNoise(var_limit=(500, 500), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(180, 180), p=1), A.GaussNoise(var_limit=(500, 500), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(225, 225), p=1), A.GaussNoise(var_limit=(500, 500), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(270, 270), p=1), A.GaussNoise(var_limit=(500, 500), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(315, 315), p=1), A.GaussNoise(var_limit=(500, 500), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.GaussNoise(var_limit=(2000, 2000), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(45, 45), p=1), A.GaussNoise(var_limit=(2000, 2000), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(90, 90), p=1), A.GaussNoise(var_limit=(2000, 2000), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(135, 135), p=1), A.GaussNoise(var_limit=(2000, 2000), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(180, 180), p=1), A.GaussNoise(var_limit=(2000, 2000), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(225, 225), p=1), A.GaussNoise(var_limit=(2000, 2000), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(270, 270), p=1), A.GaussNoise(var_limit=(2000, 2000), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.HorizontalFlip(p=1), A.Rotate(limit=(315, 315), p=1), A.GaussNoise(var_limit=(2000, 2000), mean=0, p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.Rotate(limit=(45, 45), p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.Rotate(limit=(90, 90), p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.Rotate(limit=(135, 135), p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.Rotate(limit=(180, 180), p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.Rotate(limit=(225, 225), p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.Rotate(limit=(270, 270), p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()]),
#     A.Compose([A.Rotate(limit=(315, 315), p=1), A.Resize(height=img_size, width=img_size), A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), ToTensorV2()])
# ]

# TTA 변환 정의 - 2(kkh-7)
noise_value_min = 500
noise_value_max = 500
hf_rotate_000_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_030_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(30, 30), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_060_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(60, 60), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_090_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(90, 90), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_120_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(120, 120), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_150_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(150, 150), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_180_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(180, 180), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_210_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(210, 210), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_240_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(240, 240), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_270_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(270, 270), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_300_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(300, 300), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_330_noise_0500_0500 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(330, 330), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

noise_value_min = 2000
noise_value_max = 2000
hf_rotate_000_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_030_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(30, 30), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_060_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(60, 60), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_090_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(90, 90), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_120_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(120, 120), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_150_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(150, 150), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_180_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(180, 180), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_210_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(210, 210), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_240_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(240, 240), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_270_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(270, 270), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_300_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(300, 300), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
hf_rotate_330_noise_2000_2000 = A.Compose([
    A.HorizontalFlip(p=1),
    A.Rotate(limit=(330, 330), p=1),
    A.GaussNoise(var_limit=(noise_value_min, noise_value_max), mean=0, p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# 회전 변환 + 가우시안 노이즈 (수평 뒤집기 없이, 각도별 확률 100%)
rotate_030 = A.Compose([
    A.Rotate(limit=(30, 30), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_060 = A.Compose([
    A.Rotate(limit=(60, 60), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_090 = A.Compose([
    A.Rotate(limit=(90, 90), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_120 = A.Compose([
    A.Rotate(limit=(120, 120), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_150 = A.Compose([
    A.Rotate(limit=(150, 150), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_180 = A.Compose([
    A.Rotate(limit=(180, 180), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_210 = A.Compose([
    A.Rotate(limit=(210, 210), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_240 = A.Compose([
    A.Rotate(limit=(240, 240), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_270 = A.Compose([
    A.Rotate(limit=(270, 270), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_300 = A.Compose([
    A.Rotate(limit=(300, 300), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
rotate_330 = A.Compose([
    A.Rotate(limit=(330, 330), p=1),
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# 새로운 TTA 변환을 리스트에 추가
tta_transforms = [
    hf_rotate_000_noise_0500_0500,
    hf_rotate_030_noise_0500_0500,
    hf_rotate_060_noise_0500_0500,
    hf_rotate_090_noise_0500_0500,
    hf_rotate_120_noise_0500_0500,
    hf_rotate_150_noise_0500_0500,
    hf_rotate_180_noise_0500_0500,
    hf_rotate_210_noise_0500_0500,
    hf_rotate_240_noise_0500_0500,
    hf_rotate_270_noise_0500_0500,
    hf_rotate_300_noise_0500_0500,
    hf_rotate_330_noise_0500_0500,
    hf_rotate_000_noise_2000_2000,
    hf_rotate_030_noise_2000_2000,
    hf_rotate_060_noise_2000_2000,
    hf_rotate_090_noise_2000_2000,
    hf_rotate_120_noise_2000_2000,
    hf_rotate_150_noise_2000_2000,
    hf_rotate_180_noise_2000_2000,
    hf_rotate_210_noise_2000_2000,
    hf_rotate_240_noise_2000_2000,
    hf_rotate_270_noise_2000_2000,
    hf_rotate_300_noise_2000_2000,
    hf_rotate_330_noise_2000_2000,
    rotate_030,
    rotate_060,
    rotate_090,
    rotate_120,
    rotate_150,
    rotate_180,
    rotate_210,
    rotate_240,
    rotate_270,
    rotate_300,
    rotate_330
]

# 데이터셋 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

    def get_labels(self):
        return self.df[:, 1]

# 모델 불러오기
MODEL_PATH = '/dongim/best_model.pth'
# device = torch.device('cpu')
model = timm.create_model('efficientnet_b4', pretrained=False, num_classes=17).to(device)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# 이미지 로드 함수
def load_image(image_path, transform):
    image = Image.open(image_path).convert('RGB')
    image = transform(image=np.array(image))['image']
    image = image.unsqueeze(0)  # 배치 차원 추가
    return image


# 테스트 이미지 경로 설정
TEST_IMAGE_PATH = '/dongim/data/test'
image_paths = [os.path.join(TEST_IMAGE_PATH, img) for img in os.listdir(TEST_IMAGE_PATH) if img.endswith(('png', 'jpg', 'jpeg'))]

# TTA 적용하여 이미지 분류 수행
i = 0
results = []
confidence_results = []
def trimmed_mean(predictions, proportion_to_cut=0.11):
    """
    Trimmed mean, excluding the extreme values from both ends
    """
    sorted_predictions = np.sort(predictions, axis=0)
    n = len(sorted_predictions)
    trimmed_predictions = sorted_predictions[int(n*proportion_to_cut):int(n*(1-proportion_to_cut))]
    return np.mean(trimmed_predictions, axis=0)
for image_path in image_paths:
    i += 1
    tta_predictions = []
    for tta_transform in tta_transforms:
        image = load_image(image_path, tta_transform).to(device)
        with torch.no_grad():
            output = model(image)
            prediction = F.softmax(output, dim=1)
            tta_predictions.append(prediction.cpu().numpy())
    
    # TTA 결과 평균
    # avg_prediction = np.mean(tta_predictions, axis=0)
    # TTA 결과 트리밍된 평균
    avg_prediction = trimmed_mean(tta_predictions, proportion_to_cut=0.11)
    
    predicted_class = np.argmax(avg_prediction, axis=1).item()
    confidence_score = np.max(avg_prediction, axis=1).item()
    
    results.append((os.path.basename(image_path), predicted_class))
    
    # confidence score가 0.9 이상인 경우 따로 저장
    if confidence_score >= 0.9:
        confidence_results.append((confidence_score, os.path.basename(image_path), predicted_class))
    
    print(i, f'Image: {image_path}, Predicted class: {predicted_class}, Confidence: {confidence_score:.4f}')


1 Image: /dongim/data/test/fdcd392ca7d3ba10.jpg, Predicted class: 8, Confidence: 1.0000
2 Image: /dongim/data/test/e18d8c0f27017a07.jpg, Predicted class: 5, Confidence: 0.9982
3 Image: /dongim/data/test/97704e39978cd283.jpg, Predicted class: 7, Confidence: 0.2966
4 Image: /dongim/data/test/73d4659e125ce2ef.jpg, Predicted class: 10, Confidence: 1.0000
5 Image: /dongim/data/test/32bdb23799758ad2.jpg, Predicted class: 2, Confidence: 1.0000
6 Image: /dongim/data/test/2394b085e2f8056a.jpg, Predicted class: 11, Confidence: 1.0000
7 Image: /dongim/data/test/83f9ccf8714e3478.jpg, Predicted class: 3, Confidence: 0.6166
8 Image: /dongim/data/test/77e908968cddf38a.jpg, Predicted class: 4, Confidence: 0.5369
9 Image: /dongim/data/test/15d368bbb994e331.jpg, Predicted class: 4, Confidence: 0.8683
10 Image: /dongim/data/test/0f279185fdc162fc.jpg, Predicted class: 9, Confidence: 0.9934
11 Image: /dongim/data/test/781f87e2ec698df4.jpg, Predicted class: 7, Confidence: 1.0000
12 Image: /dongim/data/test/

In [16]:
# # 결과 저장
# results_sorted = sorted(results, key=lambda x: x[0])
# df_sorted = pd.DataFrame(results_sorted, columns=["ID", "target"])
# df_sorted.to_csv("/dongim/submission/tta_predictions_bestmodel(0809).csv", index=False)

# # confidence score가 0.9 이상인 데이터 저장
# confidence_sorted = sorted(confidence_results, key=lambda x: x[1])
# df_confidence_sorted = pd.DataFrame(confidence_sorted, columns=["Confidence Score", "ID", "target"])
# df_confidence_sorted.to_csv("/dongim/submission/high_confidence_predictions.csv", index=False)

In [17]:
# # confidence score가 0.99 이상인 데이터 필터링
# df_high_confidence = df_confidence_sorted[df_confidence_sorted['Confidence Score'] >= 0.99]

# # 필터링된 데이터를 새로운 CSV 파일로 저장
# df_high_confidence.to_csv("/dongim/submission/high_confidence_predictions_99.csv", index=False)

In [21]:
# import shutil

# destination_folder = '/dongim/data/test_confidence_99'
# # 필터링된 데이터에서 ID값만 추출
# high_confidence_ids = df_high_confidence['ID'].tolist()

# # 원본 폴더에서 이미지 추출 및 복사
# source_folder = '/dongim/data/test'

# for image_id in high_confidence_ids:
#     # 각 ID에 해당하는 이미지 파일 경로 생성
#     source_image_path = os.path.join(source_folder, image_id)
    
#     if os.path.exists(source_image_path):
#         # 이미지 파일을 새로운 폴더로 복사
#         shutil.copy(source_image_path, destination_folder)
#         print(f'Copied: {source_image_path} to {destination_folder}')
#     else:
#         print(f'Image not found: {source_image_path}')

# print('Copying completed.')

# # Confidence Score 컬럼 제거
# df_high_confidence_filtered = df_high_confidence.drop(columns=['Confidence Score'])

# # 필터링된 데이터를 새로운 CSV 파일로 저장
# df_high_confidence_filtered.to_csv("/dongim/submission/very_high_confidence_predictions_filtered.csv", index=False)

In [ ]:
# # 성능 평가
# f1 = f1_score(all_targets, all_predictions, average='macro')
# accuracy = accuracy_score(all_targets, all_predictions)
# class_report = classification_report(all_targets, all_predictions, target_names=[str(i) for i in range(17)], output_dict=True)

In [4]:
results

results_sorted = sorted(results, key=lambda x: x[0])
results_sorted
df_sorted = pd.DataFrame(results_sorted, columns=["ID", "target"])
df_sorted.to_csv("/dongim/submission/bestmodel_tta_011.csv", index=False)


In [23]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 163029 KiB | 388639 KiB | 356195 GiB | 356195 GiB |
|       from large pool | 111476 KiB | 310181 KiB | 350211 GiB | 350211 GiB |
|       from small pool |  51553 KiB | 104032 KiB |   5984 GiB |   5983 GiB |
|---------------------------------------------------------------------------|
| Active memory         | 163029 KiB | 388639 KiB | 356195 GiB | 356195 GiB |
|       from large pool | 111476 KiB | 310181 KiB | 350211 GiB |